# 부스팅

- 머신러닝에서 부스팅은 오차를 줄이기 위해 사용되는 학습 방법. 앙상블 기법 중 하나로 부스팅은 이전 모델들이 만든 오차에 집중하여 그 오차를 보완하는 새로운 모델을 반복적으로 학습시킴

## 부스팅의 주요 단계
1. (Base Model 학습): 첫 번째 모델은 전체 데이터셋을 사용하여 학습된다. 이 모델은 예측에서 오차를 만들어냄.

2. (Error에 집중한 모델 학습): 첫 번째 모델이 만든 오차에 더 집중하는 두 번째 모델을 학습한다. 두 번째 모델은 오차에 가중치를 부여하여 예측 모델을 보완

3. (반복): 이후에도 계속해서 오차에 집중하는 모델을 학습하고 각 모델은 이전 모델들이 만든 오차를 보완하며, 전체 앙상블은 강력한 예측 성능을 갖게 됨

## 부스팅의 특징
- 가중치 부여: 각 모델은 이전 모델들이 만든 오차에 가중치를 부여하여 학습된다.

- Adaptive Boosting (AdaBoost), Gradient Boosting (GBM), XGBoost, LightGBM, CatBoost 등 여러 부스팅 알고리즘이 있다.

## 부스팅의 장점
- 높은 성능: 일반적으로 부스팅은 높은 예측 성능을 제공합니다.

- Overfitting 방지: 오차에 집중하여 학습하므로 과적합을 방지하는 효과가 있습니다.

- 다양한 데이터 패턴 학습: 각 모델이 이전 모델의 오차를 보완하면서 다양한 데이터 패턴을 학습합니다.

**부스팅 예시:**

1. **첫 번째 모델 (Decision Tree):**
   - 전체 데이터셋을 사용하여 학습

2. **두 번째 모델 (Decision Tree):**
   - 첫 번째 모델이 만든 오차에 가중치를 부여하여 학습

3. **세 번째 모델 (Decision Tree):**
   - 두 번째 모델이 만든 오차에 가중치를 부여하여 학습

4. **반복...**

### 예제

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/머신러닝과 딥러닝/data/total_netflix_2023 new.csv')
df

,Title,Available Globally?,Release Date,Hours Viewed,Number of Ratings,Rating,Genre,Key Words,Description
0,The Night Agent: Season 1,Yes,2023-03-23,812100000,7696.0,6.0,"Biography, Drama, History","persian empire,empire,5th century b.c.,achaeme...",The film follows headstrong Ginny who meets Su...
1,Ginny & Georgia: Season 2,Yes,2023-01-05,665100000,5216.0,5.7,"Comedy, Drama,Romance","producer,three word title,headstrong,arranged ...","Betrothed against her will to King George, you..."
2,The Glory: Season 1 // 더 글로리: 시즌 1,Yes,2022-12-30,622800000,11869.0,8.4,['Short'],"prequel,queen,historical,england,queen charlot...",A holiday romantic comedy that captures the ra...
3,Wednesday: Season 1,Yes,2022-11-23,507700000,50077.0,7.4,"['Drama', 'History', 'Romance']","christmas,coming out,holidays,lesbian relation...",The film follows headstrong Ginny who meets Su...
4,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04,503000000,45624.0,6.6,"['Comedy', 'Romance']","telenovela,drug trafficking,cartel,femme fatal...","While fleeing from dangerous assailants, an as..."
...,...,...,...,...,...,...,...,...,...
6781,Missing Link (2019),No,2020-08-21,4800000,10474.0,3.2,['Comedy'],"reboot,young girl,young,love,shampoo",Comedian Rob Schneider gives viewers a look at...
6782,Molang: Season 3,No,2022-04-27,4800000,3551.0,6.0,"['Comedy', 'Music']","baking competition,baking bread,baker,food tas...",Comedian Rory Scovel storms the stage in Atlan...
6783,Monzón: A Knockout Blow: Season 1 // Monzón: T...,No,2018-11-09,4800000,3468.0,6.4,"['Action', 'Crime', 'Drama']","life,conspiracy thriller,conspiracy,robots,ali...",RuPaul searches for America&apos;s next drag s...
6784,Mortel: Season 1,Yes,2022-06-18,4800000,1814.0,7.8,['Comedy'],non fiction,"Russell Brand takes on Icons, corporations, co..."


In [ ]:
df.head()

,Title,Available Globally?,Release Date,Hours Viewed,Number of Ratings,Rating,Genre,Key Words,Description
0,The Night Agent: Season 1,Yes,2023-03-23,812100000,7696.0,6.0,"Biography, Drama, History","persian empire,empire,5th century b.c.,achaeme...",The film follows headstrong Ginny who meets Su...
1,Ginny & Georgia: Season 2,Yes,2023-01-05,665100000,5216.0,5.7,"Comedy, Drama,Romance","producer,three word title,headstrong,arranged ...","Betrothed against her will to King George, you..."
2,The Glory: Season 1 // 더 글로리: 시즌 1,Yes,2022-12-30,622800000,11869.0,8.4,['Short'],"prequel,queen,historical,england,queen charlot...",A holiday romantic comedy that captures the ra...
3,Wednesday: Season 1,Yes,2022-11-23,507700000,50077.0,7.4,"['Drama', 'History', 'Romance']","christmas,coming out,holidays,lesbian relation...",The film follows headstrong Ginny who meets Su...
4,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04,503000000,45624.0,6.6,"['Comedy', 'Romance']","telenovela,drug trafficking,cartel,femme fatal...","While fleeing from dangerous assailants, an as..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6786 entries, 0 to 6785
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                6786 non-null   object
 1   Available Globally?  6786 non-null   object
 2   Release Date         6786 non-null   object
 3   Hours Viewed         6786 non-null   object
 4   Number of Ratings    6786 non-null   object
 5   Rating               6786 non-null   object
 6   Genre                6786 non-null   object
 7   Key Words            6786 non-null   object
 8   Description          6786 non-null   object
dtypes: object(9)
memory usage: 477.3+ KB


In [ ]:
df.drop(['Key Words', 'Available Globally?','Release Date','Description'], axis = 1,inplace=True)

KeyError: ignored

In [ ]:
# Rating이나 Number of Ratings이 없는 데이터는 사용할 수 없으므로 해당 행 삭제
df = df.dropna(subset=['Rating', 'Number of Ratings'])

In [ ]:
# Rating과 Number of Ratings을 수치형으로 변환
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df['Number of Ratings'] = pd.to_numeric(df['Number of Ratings'], errors='coerce')

In [ ]:
# LightGBM 모델 학습을 위한 데이터 준비
X = df[['Hours Viewed', 'Number of Ratings', 'Rating']]
y = df['Title']

In [ ]:
# 학습용과 테스트용으로 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# LightGBM 모델 정의 및 학습
params = {
    'objective': 'regression',  # 회귀 문제
    'metric': 'rmse',            # 평가 지표로 RMSE 사용
    'early_stopping_rounds': 50

}

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=1000, verbose=10)


# 학습된 모델을 사용하여 예측
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# 예측 결과 출력
result_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(result_df.head())

TypeError: ignored

# 스태킹
- 스태킹(Stacking)은 여러 머신러닝 모델의 예측 결과를 결합하여 높은 성능의 모델을 만들기 위한 앙상블(Ensemble) 기법 중 하나이다. 스태킹은 여러 모델이 만들어낸 예측 값을 취합하여 이를 다시 하나의 메타 모델에 입력으로 사용하는 방식으로 동작


## 스태킹의 주요 단계
1. (Base Models): 다양한 머신러닝 모델들을 사용하여 데이터를 학습시키고 각 모델은 독립 변수를 사용하여 종속 변수를 예측

2. (Meta Model): 첫 번째 단계에서 생성된 예측 값을 모은 후, 이를 독립 변수로 사용하여 하나의 메타 모델(또는 앙상블 모델)을 학습시킨다. 이 메타 모델은 개별 모델들의 예측을 토대로 최종 예측을 수행하는 역할함

3. 최종 예측: 학습된 메타 모델을 사용하여 새로운 데이터에 대한 최종 예측을 수행

## 스태킹의 장점
- 고수준의 성능: 다양한 모델의 다양성을 활용하여 보다 뛰어난 성능을 달성할 수 있습니다.

- 적응력: 다양한 유형의 모델을 함께 사용함으로써 다양한 데이터 패턴에 대응할 수 있습니다.

- 일반화 성능 향상: 과적합을 방지하고 일반화 성능을 향상시킬 수 있습니다.

**스태킹 예시:**

1. **첫 번째 단계 (Base Models):**
   - 결정 트리
   - 랜덤 포레스트
   - 서포트 벡터 머신

2. **두 번째 단계 (Meta Model):**
   - 로지스틱 회귀

3. **최종 예측:**
   - 각 모델의 예측을 모은 후 로지스틱 회귀로 최종 예측 수행